In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error

from sklearn.ensemble import VotingRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import BayesianRidge, SGDRegressor, Ridge
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [2]:
round = pd.read_csv('submission_ens_round.csv')

In [27]:
# 사용자 정의 반올림 함수 정의
def custom_round(x):
    # 소수점 셋째 자리 구하기
    x_scaled = x * 100
    decimal_part = x_scaled - tf.floor(x_scaled)
    integer_part = tf.floor(x_scaled) / 100

    # 규칙에 따라 반올림 처리
    rounded_value = tf.where(decimal_part > 0.5, tf.math.ceil(x_scaled) / 100,
                             tf.where(decimal_part < 0.5, tf.math.floor(x_scaled) / 100, x))
    return rounded_value

In [38]:
tf_tensor = tf.convert_to_tensor(round['FloodProbability'])
result = custom_round(tf_tensor)
result_numpy = result.numpy()
result_df = pd.DataFrame({'value': result_numpy})
result_df
round['FloodProbability'] = result_df['value']

True

In [40]:
round.to_csv('submission_ens_round2.csv', index=False)

,id,FloodProbability
0,1117957,0.580
1,1117958,0.455
2,1117959,0.450
3,1117960,0.470
4,1117961,0.470
...,...,...
745300,1863257,0.480
745301,1863258,0.440
745302,1863259,0.620
745303,1863260,0.550


In [43]:
round2 = round.sort_values(by=['FloodProbability'])

In [2]:
# train 데이터와 test 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 타겟 변수 분리
target = 'FloodProbability'
X_train = train.drop(columns=[target,'id'])
y_train = train[target]
X_test = test.drop(columns=['id'])

In [15]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [16]:
# X는 입력 특성 데이터, y는 이산형 종속 변수(목표 변수)입니다.
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train_encoded, test_size=0.2, random_state=42) # 필요한 경우 데이터를 훈련 및 테스트 세트로 분할합니다.

# Logistic Regression 모델을 초기화합니다.
model = LogisticRegression(solver='lbfgs', multi_class='multinomial')

In [17]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

c:\Users\mm\anaconda3\envs\flood\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\mm\anaconda3\envs\flood\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mm\anaconda3\envs\flood\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined a

Accuracy: 0.05508664271753491
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00         9
           4       0.00      0.00      0.00        10
           5       0.00      0.00      0.00         5
           6       0.00      0.00      0.00        22
           7       0.00      0.00      0.00        24
           8       0.00      0.00      0.00        30
           9       0.00      0.00      0.00        22
          10       0.00      0.00      0.00        44
          11       0.00      0.00      0.00        87
          12       0.00      0.01      0.00       147
          13       0.00      0.00      0.00       120
          14       0.00      0.00      0.00       162
          15       0.00      0.00      0.00       198
          16       0.00      0.00      0.00       262
          17       0.00      0.00      0.00       3

c:\Users\mm\anaconda3\envs\flood\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
# 반올림되지 않은 원래의 타겟 값을 추정
def estimate_original_targets(y_rounded):
    estimates = []
    for y in y_rounded:
        estimates.append(np.arange(y - 0.0005, y + 0.0005, 0.0001))
    return estimates

original_target_estimates = estimate_original_targets(y_train)

In [10]:
# 다양한 회귀 모델 준비
models = [
    ('LinearRegression', LinearRegression()),
    ('Ridge', Ridge(alpha=1.0)),
    ('Lasso', Lasso(alpha=0.1))
]

# 최적의 모델을 찾기 위한 변수 초기화
best_model = None
best_score = float('inf')
best_predictions = None


In [11]:
# 다양한 회귀 모델을 학습 및 평가
for name, model in models:
    model.fit(X_train, y_train)
    predictions = model.predict(X_train)
    predictions_rounded = np.round(predictions, 4)
    score = mean_squared_error(y_train, predictions_rounded)
    
    print(f'Model: {name}, Score: {score}')
    
    if score < best_score:
        best_score = score
        best_model = model
        best_predictions = predictions_rounded
# 0.00040395088979260915

Model: LinearRegression, Score: 0.0004037003044660933
Model: Ridge, Score: 0.0004037002417534846
Model: Lasso, Score: 0.0026036604621197416


In [12]:
print(f'Best Model: {best_model}, Best Score: {best_score}')

# test 데이터에 대해 예측 수행
test_predictions = best_model.predict(X_test)
test_predictions_rounded = np.round(test_predictions, 4)

# 결과 저장
submission = pd.DataFrame({'Id': test.index, 'target': test_predictions_rounded})
submission.to_csv('submission.csv', index=False)

Best Model: Ridge(), Best Score: 0.0004037002417534846


In [9]:
# 사용자 정의 반올림 함수 정의
def custom_round(x):
    # 소수점 셋째 자리 구하기
    x_scaled = x * 100
    decimal_part = x_scaled - tf.floor(x_scaled)
    integer_part = tf.floor(x_scaled) / 100

    # 규칙에 따라 반올림 처리
    rounded_value = tf.where(decimal_part > 0.5, tf.math.ceil(x_scaled) / 100,
                             tf.where(decimal_part < 0.5, tf.math.floor(x_scaled) / 100, x))
    return rounded_value

In [10]:
custom_round(0.328)

<tf.Tensor: shape=(), dtype=float32, numpy=0.33>

In [59]:
train2 = train.copy()
BASE_FEATURES = X_test.columns
train2['sum'] = train[BASE_FEATURES].sum(axis=1)
train2 = train2.drop(columns =['id'])
train2.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability,sum
0,5,8,5,8,6,4,4,3,3,4,...,3,3,5,4,7,5,7,3,0.445,94
1,6,7,4,4,8,8,3,5,4,6,...,2,0,3,5,3,3,4,3,0.450,94
2,6,5,6,7,3,7,1,5,4,5,...,3,7,5,6,8,2,3,3,0.530,99
3,3,4,6,5,4,8,4,7,6,8,...,4,7,4,4,6,5,7,5,0.535,104
4,5,3,2,6,4,4,3,3,3,3,...,2,6,6,4,1,2,3,5,0.415,72


In [66]:
train3 = train2.sort_values(by=['FloodProbability','sum'])
train3.to_csv('train3.csv', index=False)

In [70]:
train3.FloodProbability.unique()

array([0.285, 0.315, 0.32 , 0.325, 0.33 , 0.335, 0.34 , 0.345, 0.35 ,
       0.355, 0.36 , 0.365, 0.37 , 0.375, 0.38 , 0.385, 0.39 , 0.395,
       0.4  , 0.405, 0.41 , 0.415, 0.42 , 0.425, 0.43 , 0.435, 0.44 ,
       0.445, 0.45 , 0.455, 0.46 , 0.465, 0.47 , 0.475, 0.48 , 0.485,
       0.49 , 0.495, 0.5  , 0.505, 0.51 , 0.515, 0.52 , 0.525, 0.53 ,
       0.535, 0.54 , 0.545, 0.55 , 0.555, 0.56 , 0.565, 0.57 , 0.575,
       0.58 , 0.585, 0.59 , 0.595, 0.6  , 0.605, 0.61 , 0.615, 0.62 ,
       0.625, 0.63 , 0.635, 0.64 , 0.645, 0.65 , 0.655, 0.66 , 0.665,
       0.67 , 0.675, 0.68 , 0.685, 0.69 , 0.695, 0.7  , 0.705, 0.71 ,
       0.715, 0.725])

In [21]:
# 타겟 변수 분리
target = 'FloodProbability'
X_train = train2.drop(columns=[target,'id'])
y_train = train2[target]
X_test = test.drop(columns=[target,'id'])

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import LabelEncoder

# 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

target = 'FloodProbability'
# 타겟 값을 범주형 값으로 인코딩
le = LabelEncoder()
train['target_encoded'] = le.fit_transform(train[target])

filtered_train = train[train[target].isin([0.285, 0.315, 0.32 , 0.325, 0.33 , 0.335, 0.34 , 0.345, 0.35 ,0.355, 0.36])]

train2 = filtered_train.drop(columns=['id'])
test2 = test.drop(columns=['id'])


# # 타겟 값을 구간으로 변환
# n_bins = 83  # 원하는 구간의 수
# kbins = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
# train2['target_binned'] = kbins.fit_transform(train2[[target]])

# # 독립 변수와 종속 변수 분리
# X_train = train2.drop(columns=[target, 'target_binned']).values
# y_train = train2['target_binned'].values
# X_test = test2.values

# # 학습 데이터와 검증 데이터 분리
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=35)

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# 타겟 값이 이미 구간화되어 있는 경우, 범주형 값으로 변환
y_train = train2['target_encoded'].values  # 이미 구간화된 타겟 값을 사용

# 독립 변수 분리
X_train = train2.drop(columns=[target,'target_encoded']).values
X_test = test2.values

# 데이터 샘플링 (예: 전체 데이터의 30%만 사용)
# X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=0.3, random_state=42)

# 학습 데이터와 검증 데이터 분리
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_sample, y_train_sample, test_size=0.2, random_state=42)

# 분류 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_split, y_train_split)

# 검증 데이터에 대한 예측
y_val_pred = model.predict(X_val)

# 성능 평가
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

Validation Accuracy: 0.05940844986433704
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         2
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         8
           7       0.00      0.00      0.00        18
           8       0.00      0.00      0.00        30
           9       0.00      0.00      0.00        24
          10       0.00      0.00      0.00        36
          11       0.00      0.00      0.00        60
          12       0.00      0.00      0.00        94
          13       0.00      0.00      0.00        73
          14       0.00      0.00      0.00       142
          15       0.00      0.00      0.00       133
          16       0.00      0.00      0.00       152
          17       0.17      0.00      0

c:\Users\mm\anaconda3\envs\flood\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mm\anaconda3\envs\flood\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mm\anaconda3\envs\flood\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 타겟 값 구간 분포 시각화
plt.figure(figsize=(10, 6))
sns.countplot(x=train['target_binned'])
plt.title('Target Binned Distribution')
plt.xlabel('Target Binned')
plt.ylabel('Count')
plt.show()

# 검증 데이터 예측 값 분포 시각화
plt.figure(figsize=(10, 6))
sns.countplot(x=y_val_pred)
plt.title('Validation Predictions Distribution')
plt.xlabel('Predicted Target Binned')
plt.ylabel('Count')
plt.show()

In [ ]:
# 데이터 불러오기
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# 입력과 출력 데이터 분할
X_train = data_train.drop(['FloodProbability', 'id'], axis=1).values  # 훈련 데이터 입력 특성
y_train = data_train['FloodProbability'].values  # 훈련 데이터 타겟 변수

X_test = data_test.drop(['id'], axis=1).values  # 테스트 데이터 입력 특성

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # 출력 뉴런 하나 (홍수 발생 가능성)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 훈련
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, validation_split=0.2)

# 테스트 데이터의 타겟 변수 예측
y_pred = model.predict(X_test_scaled)

# 예측 결과를 데이터프레임으로 변환
submission_df = pd.DataFrame({'id': data_test['id'], 'FloodProbability': y_pred.flatten()})

# CSV 파일로 저장
submission_df.to_csv('submission3.csv', index=False)

In [ ]:
round = pd.read_csv('submission_ens_round.csv')

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [18]:
# Test Loss: 0.0003681324888020754
# Test MAE: 0.015425443649291992
# 데이터 불러오기
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# 입력과 출력 데이터 분할
X_train = data_train.drop(['FloodProbability', 'id'], axis=1).values  # 훈련 데이터 입력 특성
y_train = data_train['FloodProbability'].values  # 훈련 데이터 타겟 변수

X_test = data_test.drop(['id'], axis=1).values  # 테스트 데이터 입력 특성

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # 출력 뉴런 하나 (홍수 발생 가능성)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 훈련
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# 테스트 데이터의 타겟 변수 예측
y_pred = model.predict(X_test_scaled)

# 예측 결과를 데이터프레임으로 변환
submission_df = pd.DataFrame({'id': data_test['id'], 'FloodProbability': y_pred.flatten()})

# CSV 파일로 저장
submission_df.to_csv('submission.csv', index=False)

#0.860

Epoch 1/50
27949/27949 [==============================] - 35s 1ms/step - loss: 6.6095e-04 - mae: 0.0176 - val_loss: 3.7358e-04 - val_mae: 0.0149
Epoch 2/50
27907/27949 [============================>.] - ETA: 0s - loss: 3.7133e-04 - mae: 0.0152

KeyboardInterrupt: 

In [20]:
from keras.models import Sequential
from keras.layers import Dense


# 입력과 출력 데이터 나누기
X = data.drop(['FloodProbability','id'], axis=1).values  # 입력 특성
y = data['FloodProbability'].values  # 출력 타겟

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 훈련, 검증, 테스트 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=35)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=35)

# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))  # 더 많은 뉴런 추가
model.add(Dense(64, activation='relu'))   # 추가적인 레이어
model.add(Dense(1))  # 출력 뉴런 하나 (홍수 발생 가능성)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 훈련
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))

# 모델 평가
loss, mae = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/30
24456/24456 [==============================] - 36s 1ms/step - loss: 6.5333e-04 - mae: 0.0175 - val_loss: 3.7532e-04 - val_mae: 0.0156
Epoch 2/30
24456/24456 [==============================] - 35s 1ms/step - loss: 3.7268e-04 - mae: 0.0152 - val_loss: 3.6519e-04 - val_mae: 0.0152
Epoch 3/30
24456/24456 [==============================] - 35s 1ms/step - loss: 3.6843e-04 - mae: 0.0151 - val_loss: 3.7696e-04 - val_mae: 0.0156
Epoch 4/30
24456/24456 [==============================] - 36s 1ms/step - loss: 3.6711e-04 - mae: 0.0151 - val_loss: 3.5927e-04 - val_mae: 0.0148
Epoch 5/30
24456/24456 [==============================] - 36s 1ms/step - loss: 3.6605e-04 - mae: 0.0150 - val_loss: 3.6076e-04 - val_mae: 0.0149
Epoch 6/30
24456/24456 [==============================] - 35s 1ms/step - loss: 3.6534e-04 - mae: 0.0150 - val_loss: 3.6253e-04 - val_mae: 0.0149
Epoch 7/30
24456/24456 [==============================] - 35s 1ms/step - loss: 3.6459e-04 - mae: 0.0150 - val_loss: 3.6094e-04 - v

In [3]:
# 입력과 출력 데이터 분할
X_train = data_train.drop(['FloodProbability', 'id'], axis=1).values  # 훈련 데이터 입력 특성
y_train = data_train['FloodProbability'].values  # 훈련 데이터 타겟 변수

X_test = data_test.drop(['id'], axis=1).values  # 테스트 데이터 입력 특성

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(128, activation='relu'))  # 더 많은 뉴런 추가
model.add(Dense(64, activation='relu'))   # 추가적인 레이어
model.add(Dense(1))  # 출력 뉴런 하나 (홍수 발생 가능성)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 훈련
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# 테스트 데이터의 타겟 변수 예측
y_pred = model.predict(X_test_scaled)

# 예측 결과를 데이터프레임으로 변환
submission_df = pd.DataFrame({'id': data_test['id'], 'FloodProbability': y_pred.flatten()})

# CSV 파일로 저장
submission_df.to_csv('submission2.csv', index=False)


Epoch 1/50
27949/27949 [==============================] - 41s 1ms/step - loss: 6.0796e-04 - mae: 0.0172 - val_loss: 3.9237e-04 - val_mae: 0.0160
Epoch 2/50
27949/27949 [==============================] - 39s 1ms/step - loss: 3.7322e-04 - mae: 0.0152 - val_loss: 3.8872e-04 - val_mae: 0.0150
Epoch 3/50
27949/27949 [==============================] - 39s 1ms/step - loss: 3.6918e-04 - mae: 0.0151 - val_loss: 3.6249e-04 - val_mae: 0.0151
Epoch 4/50
27949/27949 [==============================] - 39s 1ms/step - loss: 3.6679e-04 - mae: 0.0151 - val_loss: 3.6367e-04 - val_mae: 0.0152
Epoch 5/50
27949/27949 [==============================] - 38s 1ms/step - loss: 3.6489e-04 - mae: 0.0150 - val_loss: 3.6551e-04 - val_mae: 0.0153
Epoch 6/50
27949/27949 [==============================] - 39s 1ms/step - loss: 3.6376e-04 - mae: 0.0150 - val_loss: 3.5923e-04 - val_mae: 0.0148
Epoch 7/50
27949/27949 [==============================] - 39s 1ms/step - loss: 3.6278e-04 - mae: 0.0150 - val_loss: 3.5992e-04 - v